In [ ]:
!pip install transformers datasets evaluate
!pip install scikit-learn numpy pandas matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [ ]:
from google.colab  import files

uploaded = files.upload()

Saving Fake.csv to Fake.csv


In [ ]:
from google.colab  import files

uploaded = files.upload()

Saving True.csv to True.csv


In [ ]:
import pandas as pd

true_df = pd.read_csv("True.csv")
fake_df = pd.read_csv("Fake.csv")

true_df['label'] = 1
fake_df['label'] = 0

df = pd.concat([true_df, fake_df]).reset_index(drop=True)
df = df[['text', 'label']]
df = df.dropna()

df = df.sample(frac=1).reset_index(drop=True)  # Shuffle
df.head()


,text,label
0,"Donald Trump s White House is in chaos, and th...",0
1,Now that Donald Trump is the presumptive GOP n...,0
2,Mike Pence is a huge homophobe. He supports ex...,0
3,SAN FRANCISCO (Reuters) - California Attorney ...,1
4,Twisted reasoning is all that comes from Pelos...,0


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42)


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)


In [ ]:
import torch

class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) for key, val in self.encodings.items()
        } | {'labels': torch.tensor(self.labels[idx])}

    def __len__(self):
        return len(self.labels)

train_dataset = FakeNewsDataset(train_encodings, train_labels)
test_dataset = FakeNewsDataset(test_encodings, test_labels)


In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip install --upgrade transformers # Ensure transformers is up-to-date
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Updated TrainingArguments with correct parameter names
training_args = TrainingArguments(
    output_dir="./results",
    # Replace 'evaluation_strategy' with 'eval_strategy'
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.000000,0.001726
2,0.000100,0.001136


TrainOutput(global_step=8980, training_loss=0.00935553328293186, metrics={'train_runtime': 7193.8619, 'train_samples_per_second': 9.986, 'train_steps_per_second': 1.248, 'total_flos': 1.890084577284096e+16, 'train_loss': 0.00935553328293186, 'epoch': 2.0})

In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_score
from sklearn.metrics import roc_curve  # Import roc_curve
from torch.utils.data import DataLoader

# ... (rest of your code) ...

# Assuming you want to use your GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make sure model is in eval mode and moved to the device
model.eval()
model.to(device)  # Move model to the device

# Create the DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=8)  # Define test_dataloader

# To collect true and predicted labels and probabilities
all_preds = []
all_labels = []
all_probs = []  # Store probabilities for AUC and EER

# Turn off gradient calculations for inference
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)[:, 1]  # Probabilities for class 1 (Fake)
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

# ✅ Evaluation
print("Classification Report:\n")
print(classification_report(all_labels, all_preds, target_names=["Real", "Fake"]))

print("Confusion Matrix:\n")
print(confusion_matrix(all_labels, all_preds))

print("Accuracy Score:", accuracy_score(all_labels, all_preds))

# Calculate AUC
auc_score = roc_auc_score(all_labels, all_probs)
print("AUC Score:", auc_score)

# Calculate Precision (for class 1 - Fake)
precision = precision_score(all_labels, all_preds)
print("Precision:", precision)

# Calculate EER
fpr, tpr, thresholds = roc_curve(all_labels, all_probs)
eer_threshold = thresholds[abs(fpr - (1 - tpr)).argmin()]
eer = fpr[abs(fpr - (1 - tpr)).argmin()]
print("EER:", eer)
print("EER Threshold:", eer_threshold)

Classification Report:

              precision    recall  f1-score   support

        Real       1.00      1.00      1.00      4669
        Fake       1.00      1.00      1.00      4311

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

Confusion Matrix:

[[4669    0]
 [   1 4310]]
Accuracy Score: 0.9998886414253898
AUC Score: 0.9999768979214538
Precision: 1.0
EER: 0.0
EER Threshold: 0.9999515
